In [9]:
import pandas as pd
import jieba as jb

In [10]:
clear_data=pd.read_csv('clear_data.csv')

In [11]:
clear_data.head()

,Unnamed: 0,Unnamed: 0.1,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing
0,0,11800,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN
1,1,11801,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN
2,2,11802,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN
3,3,11803,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN
4,4,11804,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN


In [12]:
test1=clear_data.title[1]
test1

'BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.... BBC记者表示现场伤亡严重.... '

In [13]:
res=jb.cut(test1)

In [14]:
','.join(res)

'BBC,推送,：, ,7,点,25,分,左右,泰国,曼谷,市中心,发生,大,爆炸,...., ,在,市中心,旅游景点,四面佛,附近,...., ,BBC,记者,表示,现场,伤亡,严重,...., '